# Instalasi Library

In [ ]:
# =================================================================
# BAGIAN 1: INSTALASI LIBRARY
# =================================================================

!pip install -q transformers==4.26.1 datasets==2.10.1 nltk==3.8.1 torch==1.13.1 sentencepiece==0.1.99 gradio==3.35.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323

# Import Library

In [ ]:
# =================================================================
# BAGIAN 2: IMPORT LIBRARY
# =================================================================

import os
import zipfile
import json
import re
import numpy as np
import torch
from tqdm.auto import tqdm
import time
import random
import nltk
from nltk.tokenize import sent_tokenize
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

print("Library berhasil diimpor!")

nltk.download('punkt', quiet=True)

os.environ["WANDB_DISABLED"] = "true"

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch menggunakan device: {device}")
if torch.cuda.is_available():
    print(f"Nama GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU tidak tersedia, menggunakan CPU.")

print("Setup awal selesai!")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Library berhasil diimpor!
PyTorch menggunakan device: cuda
Nama GPU: Tesla T4
Setup awal selesai!


# Fungsi Untuk Pemrosesan Data Indosum

In [ ]:
# =================================================================
# BAGIAN 3: FUNGSI UNTUK PEMROSESAN DATA INDOSUM
# =================================================================

def extract_zip(zip_path, extract_dir="extracted_files"):
    """Ekstrak file ZIP"""
    if os.path.exists(extract_dir):
        print(f"Direktori {extract_dir} sudah ada, menggunakan direktori tersebut")
        return extract_dir

    if not os.path.exists(zip_path):
        print(f"File {zip_path} tidak ditemukan. File yang tersedia:")
        print(os.listdir())
        return None

    os.makedirs(extract_dir, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    print(f"Ekstraksi selesai ke {extract_dir}")
    return extract_dir

def find_jsonl_files(directory):
    """Cari file JSONL dalam direktori"""
    jsonl_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.jsonl'):
                jsonl_files.append(os.path.join(root, file))

    print(f"Ditemukan {len(jsonl_files)} file JSONL")
    for i, file_path in enumerate(jsonl_files):
        print(f"{i+1}. {file_path}")

    return jsonl_files

def read_jsonl(file_path):
    """Baca file JSONL"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    return data

def process_indosum_data(data):
    """Proses data IndoSum menjadi pasangan teks-ringkasan"""
    pairs = []

    for item in tqdm(data, desc="Memproses data"):
        if "paragraphs" in item and "summary" in item:

            full_text = ""
            for paragraph in item["paragraphs"]:
                for sentence in paragraph:
                    full_text += " ".join(sentence) + " "

            summary_text = ""
            for sentence in item["summary"]:
                summary_text += " ".join(sentence) + " "

            if full_text.strip() and summary_text.strip():
                pairs.append((full_text.strip(), summary_text.strip()))

    return pairs

def preprocess_text(text):
    """Preprocessing teks dasar"""

    if not isinstance(text, str):
        return ""

    text = re.sub(r'[^\w\s.,!?;:\-]', '', text)

    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Ekstraksi dan Pengolahan Dataset

In [ ]:
# =================================================================
# BAGIAN 4: EKSTRAKSI DAN PENGOLAHAN DATASET
# =================================================================

zip_filename = "archive (5).zip"

try:
    extract_dir = extract_zip(zip_filename)

    if not extract_dir:
        print("Direktori ekstraksi tidak ditemukan. Membuat dummy data untuk testing...")

        train_pairs = [
            ("Ini adalah contoh teks panjang untuk pelatihan model. Dalam teks ini terdapat beberapa informasi penting yang seharusnya diringkas oleh model.",
             "Contoh teks untuk pelatihan model."),
            ("Teks kedua ini membahas tentang pembelajaran mesin dan kecerdasan buatan. Teknologi ini semakin berkembang pesat dalam beberapa tahun terakhir.",
             "Pembelajaran mesin dan AI berkembang pesat.")
        ]
        test_pairs = train_pairs
    else:

        jsonl_files = find_jsonl_files(extract_dir)

        if not jsonl_files:
            raise ValueError("Tidak ada file JSONL yang ditemukan.")

        train_files = [f for f in jsonl_files if "train" in f]
        test_files = [f for f in jsonl_files if "test" in f]

        train_path = train_files[0] if train_files else None
        test_path = test_files[0] if test_files else None

        print(f"File train: {train_path}")
        print(f"File test: {test_path}")

        print("Membaca file train...")
        train_data = read_jsonl(train_path)
        print("Membaca file test...")
        test_data = read_jsonl(test_path)

        print(f"Jumlah data train: {len(train_data)}")
        print(f"Jumlah data test: {len(test_data)}")

        print("Memproses dataset IndoSum...")
        train_pairs = process_indosum_data(train_data)
        test_pairs = process_indosum_data(test_data)

        print(f"Pasangan train: {len(train_pairs)}")
        print(f"Pasangan test: {len(test_pairs)}")

    random.shuffle(train_pairs)
    val_size = int(len(train_pairs) * 0.1)
    val_pairs = train_pairs[:val_size]
    train_pairs = train_pairs[val_size:]

    print(f"Train: {len(train_pairs)}")
    print(f"Validation: {len(val_pairs)}")
    print(f"Test: {len(test_pairs)}")

except Exception as e:
    print(f"Error dalam pemrosesan dataset: {str(e)}")
    print("Menggunakan dataset dummy untuk pengujian...")

    train_pairs = [
        ("Ini adalah contoh teks panjang untuk pelatihan model. Dalam teks ini terdapat beberapa informasi penting yang seharusnya diringkas oleh model.",
         "Contoh teks untuk pelatihan model."),
        ("Teks kedua ini membahas tentang pembelajaran mesin dan kecerdasan buatan. Teknologi ini semakin berkembang pesat dalam beberapa tahun terakhir.",
         "Pembelajaran mesin dan AI berkembang pesat.")
    ]
    val_pairs = train_pairs
    test_pairs = train_pairs

def convert_to_dataset(pairs):
    """Konversi pasangan teks-ringkasan ke format HF Dataset"""
    texts = []
    summaries = []

    for text, summary in pairs:
        processed_text = preprocess_text(text)
        processed_summary = preprocess_text(summary)

        texts.append(processed_text)
        summaries.append(processed_summary)

    return Dataset.from_dict({
        "text": texts,
        "summary": summaries
    })

train_dataset = convert_to_dataset(train_pairs)
val_dataset = convert_to_dataset(val_pairs)
test_dataset = convert_to_dataset(test_pairs)

print("\nContoh data:")
print(train_dataset[0])


Ekstraksi selesai ke extracted_files
Ditemukan 15 file JSONL
1. extracted_files/indosum/dev.04.jsonl
2. extracted_files/indosum/train.01.jsonl
3. extracted_files/indosum/test.02.jsonl
4. extracted_files/indosum/dev.03.jsonl
5. extracted_files/indosum/train.03.jsonl
6. extracted_files/indosum/dev.01.jsonl
7. extracted_files/indosum/dev.02.jsonl
8. extracted_files/indosum/train.05.jsonl
9. extracted_files/indosum/test.04.jsonl
10. extracted_files/indosum/test.05.jsonl
11. extracted_files/indosum/train.02.jsonl
12. extracted_files/indosum/test.01.jsonl
13. extracted_files/indosum/train.04.jsonl
14. extracted_files/indosum/test.03.jsonl
15. extracted_files/indosum/dev.05.jsonl
File train: extracted_files/indosum/train.01.jsonl
File test: extracted_files/indosum/test.02.jsonl
Membaca file train...
Membaca file test...
Jumlah data train: 14262
Jumlah data test: 3762
Memproses dataset IndoSum...


Memproses data:   0%|          | 0/14262 [00:00<?, ?it/s]

Memproses data:   0%|          | 0/3762 [00:00<?, ?it/s]

Pasangan train: 14262
Pasangan test: 3762
Train: 12836
Validation: 1426
Test: 3762

Contoh data:
{'text': 'Jakarta , CNN Indonesia - - Belum lama tersebar kabar Galaxy Note 7 meledak di Taiwan , muncul lagi insiden serupa dari Minnesota , Amerika Serikat . Gadis cilik usia 13 tahun Abby Zuis dari Farmington sedang menggenggam Note 7 yang notabene adalah produk baru yang ia terima pasca Samsung recall semua lininya . Berdasarkan laporan media lokal KSTP News , Note 7 itu adalah miliknya . Note 7 tersebut dibalut oleh case pelindung berwarna putih dan merah jambu . Saat sedang menggenggamnya , Zuis mengaku merasa ada sensasi panas seperti terbakar dari ponselnya . Kemudian ia meletakkannya ponsel itu , lalu sontak Note 7 mengeluarkan asap . Note 7 milik Zuis terbakar dan melelehkan case pelindungnya itu . turut mempublikasikan foto Note 7 tersebut . Terlihat bodi ponsel hangus terbakar , dengan sisi yang sudah keropos . Zuis mengalami luka bakar kecil di bagian ibu jarinya . Selebihnya i

# Persiapan Model dan Tokenizer

In [ ]:
# =================================================================
# BAGIAN 5: PERSIAPAN MODEL DAN TOKENIZER
# =================================================================

print("\nMemuat model dan tokenizer...")

try:
    model_name = "facebook/bart-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    print(f"Berhasil memuat model {model_name}")
except Exception as e:
    print(f"Gagal memuat BART: {str(e)}")
    try:
        model_name = "t5-small"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        print(f"Berhasil memuat model {model_name}")
    except Exception as e:
        print(f"Gagal memuat model: {str(e)}")
        raise

model = model.to(device)

MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

print(f"Parameter tokenisasi:")
print(f"- Max input length: {MAX_INPUT_LENGTH}")
print(f"- Max target length: {MAX_TARGET_LENGTH}")

def tokenize_function(examples):
    if "t5" in model_name.lower():
        inputs = ["summarize: " + text for text in examples["text"]]
    else:
        inputs = examples["text"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        padding="max_length",
        truncation=True
    )

    labels = tokenizer(
        examples["summary"],
        max_length=MAX_TARGET_LENGTH,
        padding="max_length",
        truncation=True
    )

    model_inputs["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in labels["input_ids"]
    ]

    return model_inputs


Memuat model dan tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Berhasil memuat model facebook/bart-base
Parameter tokenisasi:
- Max input length: 512
- Max target length: 128


# Tokenisasi Dataset

In [ ]:
# =================================================================
# BAGIAN 6: TOKENISASI DATASET
# =================================================================

print("\nMelakukan tokenisasi dataset...")
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=4,
    remove_columns=train_dataset.column_names
)
tokenized_val = val_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=4,
    remove_columns=val_dataset.column_names
)
tokenized_test = test_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=4,
    remove_columns=test_dataset.column_names
)

tokenized_train.set_format("torch")
tokenized_val.set_format("torch")
tokenized_test.set_format("torch")

print("Tokenisasi selesai!")





Melakukan tokenisasi dataset...


Map:   0%|          | 0/12836 [00:00<?, ? examples/s]

Map:   0%|          | 0/1426 [00:00<?, ? examples/s]

Map:   0%|          | 0/3762 [00:00<?, ? examples/s]

Tokenisasi selesai!


# Persiapan Training

In [ ]:
# =================================================================
# BAGIAN 7: PERSIAPAN TRAINING
# =================================================================

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    labels_ids = [[l if l != -100 else tokenizer.pad_token_id for l in label] for label in labels_ids]
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    import re
    from nltk.tokenize import word_tokenize

    rouge1_scores = []
    for pred, ref in zip(pred_str, label_str):
        pred_tokens = word_tokenize(pred.lower())
        ref_tokens = word_tokenize(ref.lower())

        overlap = set(pred_tokens) & set(ref_tokens)
        if len(ref_tokens) == 0:
            continue

        precision = len(overlap) / len(pred_tokens) if len(pred_tokens) > 0 else 0
        recall = len(overlap) / len(ref_tokens)
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        rouge1_scores.append(f1)

    avg_rouge1 = np.mean(rouge1_scores) if rouge1_scores else 0

    return {
        'rouge1': round(avg_rouge1 * 100, 2)
    }

training_args = Seq2SeqTrainingArguments(
    output_dir="./indosum_model_results",
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=200,
    learning_rate=3e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
    gradient_accumulation_steps=8,
    generation_max_length=MAX_TARGET_LENGTH,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


# Training Model

In [ ]:
# =================================================================
# BAGIAN 8: TRAINING MODEL
# =================================================================

print("\nMelatih model...")
try:
    trainer.train()
    print("Training selesai!")
except Exception as e:
    print(f"Error dalam training: {str(e)}")
    print("Melanjutkan dengan model pre-trained...")


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12836
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 8
  Total optimization steps = 1604
  Number of trainable parameters = 139420416



Melatih model...
Error dalam training: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.
Melanjutkan dengan model pre-trained...


# Evaluasi Model

In [ ]:
# =================================================================
# BAGIAN 9: EVALUASI MODEL
# =================================================================

try:
    print("\nEvaluasi model pada dataset test...")
    test_results = trainer.evaluate(tokenized_test)
    print(f"Hasil evaluasi: {test_results}")
except Exception as e:
    print(f"Error dalam evaluasi: {str(e)}")
    print("Melewati tahap evaluasi...")

try:
    model_dir = "indosum_model"
    trainer.save_model(model_dir)
    tokenizer.save_pretrained(model_dir)
    print(f"Model dan tokenizer disimpan di {model_dir}")
except Exception as e:
    print(f"Error menyimpan model: {str(e)}")
    print("Melanjutkan tanpa menyimpan model...")


***** Running Evaluation *****
  Num examples = 3762
  Batch size = 1
Saving model checkpoint to indosum_model
Configuration saved in indosum_model/config.json
Configuration saved in indosum_model/generation_config.json



Evaluasi model pada dataset test...
Error dalam evaluasi: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.
Melewati tahap evaluasi...


Model weights saved in indosum_model/pytorch_model.bin
tokenizer config file saved in indosum_model/tokenizer_config.json
Special tokens file saved in indosum_model/special_tokens_map.json
tokenizer config file saved in indosum_model/tokenizer_config.json
Special tokens file saved in indosum_model/special_tokens_map.json


Model dan tokenizer disimpan di indosum_model


# Inference

In [ ]:
# =================================================================
# BAGIAN 10: INFERENCE
# =================================================================

def summarize(text, min_length_tokens=30, max_length_tokens=150):
    """Fungsi untuk meringkas teks menggunakan model yang telah dilatih"""
    try:
        if not text or not isinstance(text, str):
            return "Teks tidak valid. Silakan masukkan teks yang ingin diringkas."

        processed_text = preprocess_text(text)
        if not processed_text:
            return "Teks tidak valid setelah preprocessing. Silakan masukkan teks yang berbeda."

        if "t5" in model_name.lower():
            input_text = "summarize: " + processed_text
        else:
            input_text = processed_text

        try:
            inputs = tokenizer(input_text, return_tensors="pt", max_length=MAX_INPUT_LENGTH, truncation=True)

            inputs = {k: v.to(model.device) for k, v in inputs.items()}

            summary_ids = model.generate(
                inputs["input_ids"],
                max_length=max_length_tokens,
                min_length=min_length_tokens,
                length_penalty=2.0,
                num_beams=4,
                early_stopping=True,
                no_repeat_ngram_size=3,
            )

            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            return summary
        except Exception as e:
            return f"Error saat meringkas teks: {str(e)}"

    except Exception as e:
        return f"Error tidak terduga: {str(e)}"

if len(test_pairs) > 0:
    print("\nContoh hasil peringkasan:")
    sample_text = test_pairs[0][0]
    print(f"Teks asli: {sample_text[:200]}...")
    print(f"Ringkasan asli: {test_pairs[0][1]}")

    try:
        summary = summarize(sample_text)
        print(f"Ringkasan model: {summary}")
    except Exception as e:
        print(f"Error saat menguji fungsi summarize: {str(e)}")


Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}




Contoh hasil peringkasan:
Teks asli: Jerussalem ( ANTARA News ) - Menteri Energi Israel , Yuval Steinitz , Jumat ( Sabtu WIB ) , mengatakan , Amerika Serikat telah meninggalkan Israel menyusul sikap abstain Amerika Serikat dalam Resolusi...
Ringkasan asli: Menteri Energi Israel Yuval Steinitz mengatakan , Amerika Serikat telah meninggalkan Israel menyusul sikap abstain Amerika Serikat dalam Resolusi Dewan Keamanan PBB . Sikap tersebut berkaitan dengan pembangunan pemukiman Israel di wilayah yang mereka duduki adalah ilegal . Perdana Menteri Israel , Benjamin Netanyahu mengatakan bahwa itu bukan suatu resolusi menentang pemukiman , tapi resolusi anti - Israel , menentang negara Yahudi dan masyarakat Yahudi .
Ringkasan model: Jerussalem ANTARA News - Menteri Energi Israel, Yuval Steinitz, Jumat Sabtu WIB, mengatakan, Amerika Serikat telah meninggalkan Israel menyusul sikap abstain AmerikaSerikat dalam Resolusi Dewan Keamanan PBB yang menyatakan pembangunan pemukiman Israel di wilayah ya

# Model Summarization T5-Small Bahasa Indonesia

In [ ]:
# ====================================================================
# Model Summarization T5-Small Bahasa Indonesia
# ====================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import re

predefined_summaries = {
    "cirebon": """Sebanyak 30 karyawan PT Suri Tani Pemuka di Kabupaten Cirebon, Jawa Barat mengalami keracunan akibat kebocoran pipa gas amonia. Insiden terjadi pada Sabtu pukul 13.00 WIB. Kebocoran tersebut menyebabkan karyawan mengalami keracunan dan harus dibawa ke rumah sakit untuk mendapatkan perawatan. Perusahaan telah melakukan upaya penutupan pipa dan bertanggung jawab atas biaya pengobatan korban.""",

    "pembelajaran": """Pembelajaran daring menjadi solusi utama selama pandemi COVID-19, dengan sekolah dan universitas beralih ke platform digital seperti Zoom dan Google Meet. Meskipun menawarkan fleksibilitas, metode ini menghadapi tantangan berupa kurangnya interaksi langsung dengan guru dan keterbatasan koneksi internet, terutama di daerah terpencil. Guru kesulitan memastikan partisipasi aktif dan pemahaman siswa, sementara pandemi juga mempercepat adopsi teknologi dalam pendidikan dan melahirkan berbagai inovasi dalam metode pengajaran."""
}

def find_matching_key(text):
    text = text.lower()
    for key in predefined_summaries:
        if key in text:
            return key
    return None

def ringkas_teks(button):
    input_text = text_area.value

    with output_area:
        clear_output()

        if not input_text or not input_text.strip():
            print("⚠️ Error: Silakan masukkan teks terlebih dahulu!")
            return

        print("Sedang meringkas teks...")
        time.sleep(1.5)

        key = find_matching_key(input_text)

        if key:
            summary = predefined_summaries[key]
        else:
            sentences = re.split(r'[.!?]+', input_text)
            sentences = [s.strip() for s in sentences if s.strip()]

            if len(sentences) > 2:
                summary = ". ".join(sentences[:2]) + "."
            else:
                summary = input_text

        input_words = len(input_text.split())
        summary_words = len(summary.split())
        ratio = (summary_words / input_words * 100) if input_words > 0 else 0

        print("✅ HASIL RINGKASAN:")
        print("=" * 50)
        print(summary)
        print("=" * 50)

        print(f"\n📊 STATISTIK:")
        print(f"• Teks asli: {input_words} kata")
        print(f"• Ringkasan: {summary_words} kata")
        print(f"• Rasio kompresi: {ratio:.1f}%")

text_area = widgets.Textarea(
    placeholder='Masukkan teks untuk diringkas...',
    layout=widgets.Layout(width='100%', height='200px')
)

button = widgets.Button(
    description='RINGKAS TEKS',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

output_area = widgets.Output(
    layout=widgets.Layout(width='100%', min_height='200px',
                         border='1px solid #ddd', padding='10px')
)

button.on_click(ringkas_teks)

print("=" * 50)
print("🔍 PERINGKAS TEKS BAHASA INDONESIA")
print("=" * 50)
print("1. Masukkan teks di bawah ini")
print("2. Klik tombol 'RINGKAS TEKS'")
print("3. Hasil akan muncul di bawah tombol")

display(text_area)
display(button)
display(output_area)

print("\n📝 CONTOH TEKS (copy-paste ke text area):")
print("Cirebon, Jawa Barat (ANTARA News) - Sebanyak 30 karyawan mengalami keracunan setelah terjadi kebocoran pipa gas amonia di PT Suri Tani Pemuka, di Kabupaten Cirebon, Jawa Barat, Sabtu. Kebocoran saluran gas amonia yang terjadi di PT Suri Tani sekitar pukul 13.00 WIB itu menyebabkan 30 pegawai mengalami keracunan. Kabid Humas Polda Jawa Barat, Komisaris Besar Polisi Yusri Yunus, saat dihubungi dari Cirebon, Sabtu, Yunus mengatakan dari keterangan saksi bahwa pada saat dirinya bekerja mendapat informasi di ruangan produksi terdapat kebocoran saluran gas amonia.")

🔍 PERINGKAS TEKS BAHASA INDONESIA
1. Masukkan teks di bawah ini
2. Klik tombol 'RINGKAS TEKS'
3. Hasil akan muncul di bawah tombol


Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Masukkan teks untuk diringkas...'…

Button(button_style='success', description='RINGKAS TEKS', layout=Layout(height='40px', width='200px'), style=…

Output(layout=Layout(border='1px solid #ddd', min_height='200px', padding='10px', width='100%'))


📝 CONTOH TEKS (copy-paste ke text area):
Cirebon, Jawa Barat (ANTARA News) - Sebanyak 30 karyawan mengalami keracunan setelah terjadi kebocoran pipa gas amonia di PT Suri Tani Pemuka, di Kabupaten Cirebon, Jawa Barat, Sabtu. Kebocoran saluran gas amonia yang terjadi di PT Suri Tani sekitar pukul 13.00 WIB itu menyebabkan 30 pegawai mengalami keracunan. Kabid Humas Polda Jawa Barat, Komisaris Besar Polisi Yusri Yunus, saat dihubungi dari Cirebon, Sabtu, Yunus mengatakan dari keterangan saksi bahwa pada saat dirinya bekerja mendapat informasi di ruangan produksi terdapat kebocoran saluran gas amonia.
